# Synthetic Data Generation using SDV

The Synthetic Data Vault (SDV) is a Python library designed for creating tabular synthetic data. The SDV uses a variety of machine learning algorithms to learn patterns from your real data and emulate them in synthetic data.

In [1]:
%pip install sdv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.3/123.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.1/145.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: numpy
    F

## 1. Data Preparation

### Loading data

In [1]:
import sdv as sdv
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/ADNOC/with_only_context.csv")
df.head()

,ID,context
0,1,On Day 01/01/2021 Well Name is FORGE 16A [78...
1,2,On Day 01/02/2021 Well Name is FORGE 16A [78...
2,3,On Day 01/03/2021 Well Name is FORGE 16A [78...
3,4,On Day 10/22/2020 Well Name is FORGE 16A [78...
4,5,On Day 10/23/2020 Well Name is FORGE 16A [78...


### Metadata Description

The SDV requires that you provide a description of your data, also known as **metadata**.

The metadata describes the types of data that are available in every column.

In [3]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df)

#The SDV can auto detect portions of the metadata by inspecting your actual data.
print('Auto detected data:\n')
metadata

Auto detected data:



{
    "columns": {
        "ID": {
            "sdtype": "numerical"
        },
        "context": {
            "sdtype": "categorical"
        }
    },
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

In [ ]:
# metadata.detect_table_from_dataframe(
#     table_name='data',
#     data=df
# )

### Updating the data types

In [4]:
# # We can include a format string to parse the date and time components.
# metadata.update_column(
#     column_name='RPT DATE',
#     sdtype='datetime',
#     datetime_format=" %m/%d/%Y"
# )

# ID Columns: These columns do not have statistical value, as they are only used to identify rows.
metadata.update_column(
    column_name='ID',
    sdtype='id'
)

### Setting the primary key

**Primary Keys**: These keys identify every row of the table. They must be unique to the entire table and other tables may refer to them.

In [5]:
metadata.set_primary_key(
    column_name='ID'
)

Now, the metadata should be accurate. Let's validate it. If successful, the code should run without any errors.

In [8]:
metadata.validate()
# metadata.save_to_json('metadata.json')
# metadata = MultiTableMetadata.load_from_json('metadata.json')

In [9]:
metadata

{
    "primary_key": "ID",
    "columns": {
        "ID": {
            "sdtype": "id"
        },
        "context": {
            "sdtype": "categorical"
        }
    },
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

### Creating a Synthesizer

An SDV **synthesizer** is an object that you can use to create synthetic data. It learns patterns from the real data and replicates them to generate synthetic data.

In [10]:
from sdv.lite import SingleTablePreset
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata)
# synthesizer = SingleTablePreset(
#     metadata,
#     name='FAST_ML'
# )

synthesizer.fit(
    data=df
)

Use the `sample` function and pass in any number of rows to synthesize.

In [12]:
synthetic_data = synthesizer.sample(
    num_rows=100
)

synthetic_data.head()

,ID,context
0,0,On Day 11/01/2020 Well Name is FORGE 16A [78...
1,1,On Day 12/16/2020 Well Name is FORGE 16A [78...
2,2,On Day 11/09/2020 Well Name is FORGE 16A [78...
3,3,On Day 11/27/2020 Well Name is FORGE 16A [78...
4,4,On Day 11/03/2020 Well Name is FORGE 16A [78...


In [ ]:
from google.colab import files

synthetic_data.to_csv('synthetic_data_context.csv', encoding = 'utf-8-sig')
files.download('synthetic_data_context.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>